In [192]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
import dash_daq as daq


# Load the CSV file into a DataFrame
df = pd.read_csv('Student Profiles Wrangled.csv')

In [193]:
def categorize_gpa(row):
    gpa = row['GPA']
    if gpa < 1:
        return 'Less Than 1'
    elif gpa < 2 and gpa >= 1:
        return '1 - 2'
    elif gpa < 3 and gpa >= 2:
        return '2 - 3'
    elif gpa < 3.5 and gpa >= 3:
        return '3 - 3.5'
    else:
        return 'More Than 3.5'

df['GPA Category'] = df.apply(categorize_gpa, axis=1)

In [194]:
import pandas as pd
import re

def categorize_job_title(title):
    if pd.isna(title):
        return 'Not Specified'
    
    title = str(title).lower()
    
    if re.search(r'hr|human resource|recruit|talent|people ops', title):
        return 'Human Resources'
    elif re.search(r'admin|secretary|receptionist|clerk|office', title):
        return 'Administration'
    elif re.search(r'manager|director|head|lead|supervisor|vice president', title):
        return 'Management'
    elif re.search(r'account|finance|payroll', title):
        return 'Finance and Accounting'
    elif re.search(r'client|sales|service|customer', title):
        return 'Customer Service and Sales'
    elif re.search(r'operation|logistics|supply|coordinator', title):
        return 'Operations and Logistics'
    elif re.search(r'market|pr|public relation|brand|communication', title):
        return 'Marketing and Public Relations'
    elif re.search(r'teacher|lecturer|trainer|education|learning', title):
        return 'Education and Training'
    elif re.search(r'it|tech|developer|engineer|analyst', title):
        return 'Technical and IT'
    else:
        return 'Others'

def add_job_category(df):
    df['Job Category'] = df['DESIGNATION'].apply(categorize_job_title)
    return df

# Assuming your dataframe is called 'df'
df = add_job_category(df)

In [195]:
df = pd.read_csv('Student Profiles Wrangled Part 2.csv')

In [196]:
df.rename(columns={'date_diff': 'Course Duration'}, inplace=True)

In [197]:
df['Course Duration'].unique()

array([456, 455, 457, 270, 268, 274, 273,  58,  61,  51, 306,  56,   1,
        60,  53, 181, 183, 180], dtype=int64)

# Plotly Sunburst

In [198]:
import pandas as pd
import numpy as np
import plotly.express as px

# Drop rows with missing values in the relevant columns
df.dropna(subset=['COURSE', 'Age Group', 'GPA'], inplace=True)

# Filter data to include only the top 5 courses by student count
top_courses = df['COURSE'].value_counts().nlargest(5).index
df_filtered = df[df['COURSE'].isin(top_courses)]

# Add a Count column for aggregation
df_filtered['Count'] = 1

# Group by course and age group and calculate average GPA and count
df_grouped = df_filtered.groupby(['COURSE', 'Age Group']).agg(
    Count=('Count', 'sum'),
    Avg_GPA=('GPA', 'mean')
).reset_index()

# Calculate average GPA for each course
course_gpa = df_filtered.groupby('COURSE')['GPA'].mean().reset_index(name='GPA_course')

# Merge the average GPAs into the grouped dataframe
df_grouped = df_grouped.merge(course_gpa, on='COURSE')

# Remove any rows where Count is 0
df_grouped = df_grouped[df_grouped['Count'] > 0]

# Create the sunburst plot with a custom color scheme
fig = px.sunburst(df_grouped, path=['COURSE', 'Age Group'], values='Count',
                  color='Avg_GPA', 
                  color_continuous_scale='Viridis',
                  hover_data={
                      'Avg_GPA': ':.2f',
                      'GPA_course': ':.2f'
                  })

# Update traces for better visibility
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  'Course GPA: %{customdata[1]:.2f}<br>' +
                  'Age Group GPA: %{customdata[0]:.2f}',
    insidetextorientation='radial',
    textfont=dict(size=14)
)

# Update layout for better aesthetics and readability
fig.update_layout(
    title={
        'text': 'Course and Age with Average GPAs',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='black'),
    font=dict(family='Arial, sans-serif', size=16, color='black'),
    margin=dict(t=80, l=0, r=0, b=0),
    paper_bgcolor='white',
    plot_bgcolor='white',
    width=1000,  # Increase width
    height=1000  # Increase height
)
# Show the plot
fig.show()

In [206]:
fig2 = px.scatter(df, x="COURSE FEE", y="Course Duration", color="CITIZENSHIP_STATUS",size='COURSE FEE',marginal_y="violin", trendline="ols")
fig2.update_layout(title="Scatter 4",xaxis_title="GPA of Students",yaxis_title="Course Duration")
fig2.show()


In [200]:
import plotly.graph_objects as go
import pandas as pd
import textwrap

def wrap_text(text, width=15):
    return '<br>'.join(textwrap.wrap(text, width=width))

# Assuming df is your DataFrame
# If you don't have df defined, you'll need to create or load it first

# Calculate the count of data points for each course
count_by_course = df['COURSE'].value_counts().reset_index()
count_by_course.columns = ['COURSE', 'Count']
count_by_course = count_by_course.sort_values(by='Count', ascending=False)

# Reorder the DataFrame based on sorted courses
df['COURSE'] = pd.Categorical(df['COURSE'], categories=count_by_course['COURSE'], ordered=True)

# Get sorted course labels
course_labels = count_by_course['COURSE']

# Create a darker color scale
colors = ['#1a5276', '#154360', '#0f3c4d', '#0b2c3a', '#071f28', '#031016', '#000506', '#000000']

# Create the box plot
fig = go.Figure()

for i, course in enumerate(course_labels):
    course_data = df[df['COURSE'] == course]['Age']
    
    fig.add_trace(go.Box(
        y=course_data,
        name=course,
        boxpoints='all',
        jitter=0.3,
        whiskerwidth=0.2,
        marker_color=colors[i % len(colors)],
        line_color=colors[i % len(colors)],
        showlegend=False,
        notched=True
    ))

# Wrap the course labels
wrapped_labels = [wrap_text(label) for label in course_labels]

# Update layout
fig.update_layout(
    title={
        'text': "Age Distribution by Course",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
        title="Course",
        tickmode='array',
        tickvals=list(range(len(course_labels))),
        ticktext=wrapped_labels,
        tickangle=0
    ),
    yaxis=dict(
        title="Age"
    ),
    font=dict(
        size=12
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(b=150)  # Increase bottom margin to accommodate wrapped labels
)

# Update the x-axis to show all labels
fig.update_xaxes(tickangle=0, tickmode='array', tickvals=list(range(len(course_labels))), ticktext=wrapped_labels)

# Update y-axis to have a white background with light grey gridlines
fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgrey',
    zeroline=True,
    zerolinecolor='lightgrey'
)

# Return the figure object
fig.show()

In [201]:
fig = px.box(df, y="Age", facet_col="COURSE FUNDING", color="COURSE FUNDING",
             boxmode="overlay", points='all')

fig.update_traces( jitter=0.5, col=1)
fig.update_traces( jitter=0.5, col=2)
fig.update_traces( jitter=0.5, col=3)
fig.update_traces( jitter=0.5, col=3)

fig.show()

In [202]:
import plotly.express as px
fig = px.histogram(df, x='COMPLETION DATE',color="COURSE FUNDING",marginal="rug",text_auto=True)

fig.show()

In [203]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

colors = {
    'background': '#f8f9fa',
    'text': '#2c3e50',
    'primary': '#3498db',
    'secondary': '#2ecc71',
    'accent': '#e74c3c'
}

# Assume df is your DataFrame
# Make sure to load your data into df before this point

def wrap_text_every_second_space(text):
    words = text.split(' ')
    return ' '.join([f'{words[i]} {words[i+1]}<br>' if i+1 < len(words) else words[i] for i in range(0, len(words), 2)]).strip()

def wrap_text_every_second_space_2(text):
    words = text.split()
    return '<br>'.join(words)

def create_heatmap(df, x_axis, y_axis, title_suffix=''):
    heatmap_data = pd.crosstab(df[y_axis], df[x_axis])
    fig_heatmap = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='Blues',
        text=heatmap_data.values,
        texttemplate='%{text}',
        colorbar=dict(title='Count')
    ))

    fig_heatmap.update_layout(
        title=dict(text=f'{y_axis} vs {x_axis} {title_suffix}', x=0.5, xanchor='center', yanchor='top'),
        xaxis_title=x_axis,
        yaxis_title=y_axis,
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(heatmap_data.columns))),
            ticktext=[wrap_text_every_second_space(label) for label in heatmap_data.columns],
            tickangle=0
        ),
        plot_bgcolor='#ffffff',
        paper_bgcolor='#ffffff',
        font_color='#000000',
        title_font_size=12,
        legend_title_font_size=14,
        legend_font_size=12,
        margin=dict(l=40, r=40, t=40, b=40),
        height=400
    )

    return fig_heatmap
    
def create_age_distribution_pie(df):
    age_counts = df['COURSE FUNDING'].value_counts()
    blue_colors = px.colors.sequential.Blues[::-2]  # Reverse the color scale
    fig = go.Figure(data=[go.Pie(
        labels=age_counts.index, 
        values=age_counts.values, 
        hole=0.4,
        marker=dict(colors=blue_colors)
    )])
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="top",
            y=0.001,
            xanchor="center",
            x=0.5,
            font=dict(size=12)
        ),
        margin=dict(l=20, r=20, t=0, b=20),
        height=400,
        autosize=True
    )
    return fig

def create_nationality_histogram(df):
    # Calculate the total counts per COURSE and sort them
    total_counts = df['COURSE'].value_counts().reset_index()
    total_counts.columns = ['COURSE', 'counts']
    total_counts = total_counts.sort_values(by='counts', ascending=False)
    
    # Assign gradient colors
    blue_colors = px.colors.sequential.Blues[::-1]
    
    # Merge sorted counts back with the original dataframe
    df = df.merge(total_counts[['COURSE', 'counts']], on='COURSE')
    
    # Reorder courses in the dataframe according to sorted counts
    df['COURSE'] = pd.Categorical(df['COURSE'], categories=total_counts['COURSE'], ordered=True)
    
    fig = px.histogram(
        df, 
        x="COURSE",
        color='COURSE',
        category_orders={'COURSE': total_counts['COURSE'].tolist()},
        color_discrete_sequence=blue_colors
    )
    
    fig.update_layout(
        showlegend=False,
        title="Gender Distribution by Course",
        xaxis_title="Course",
        yaxis_title="Count",
        height=400,
        margin=dict(l=40, r=40, t=40, b=40),
        plot_bgcolor='#ffffff',
        paper_bgcolor='#ffffff',
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(df['COURSE'].unique()))),
            ticktext=[wrap_text_every_second_space_2(label) for label in df['COURSE'].unique()],
            tickangle=0
        )
    )
    
    return fig



age_groups = df['Age Group'].unique()
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_columns = [col for col in categorical_columns if col not in [
    'STUDENT ID', 'NAME OF QUALIFICATION AND INSTITUTION', 'DATE ATTAINED HIGHEST QUALIFICATION',
    'DESIGNATION', 'INTAKE NO', 'COMMENCEMENT DATE', 'FULL-TIME OR PART-TIME', 'COURSE FUNDING',
    'PAYMENT MODE', 'DOB', 'COMPLETION DATE', 'SALUTATION', 'HIGHEST QUALIFICATION'
]]

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Rejey DAVI Dashboard", className="display-4 text-primary mb-4")
        ], width=12, className='text-center')
    ]),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Filters", className="card-title mb-4", style={'text-align': 'center','text-decoration': 'underline'}),
                    html.H6("X axis", className='text-muted mt-4'),
                    dcc.Dropdown(
                        id='x-axis',
                        options=[{'label': cat, 'value': cat} for cat in categorical_columns],
                        value='COURSE',
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Y axis", className='text-muted mt-4'),
                    dcc.Dropdown(
                        id='y-axis',
                        options=[{'label': cat, 'value': cat} for cat in categorical_columns],
                        value='Job Category',
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Job Category", className="text-muted"),
                    dcc.Dropdown(
                        id='Job-category',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': category, 'value': category} for category in df['Job Category'].unique()],
                        value='ALL',
                        multi=True,
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Nationality", className='text-muted mt-4'),
                    dcc.Dropdown(
                        id='radio-buttons',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                        value='ALL',
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Gender", className='text-muted mt-4'),
                    dbc.RadioItems(
                        id='Gender',
                        options=[
                            {'label': 'All', 'value': 'ALL'},
                            {'label': 'Male', 'value': 'M'},
                            {'label': 'Female', 'value': 'F'}
                        ],
                        value='ALL',
                        inline=True,
                        className="mb-3"
                    )
                ])
            ], className="shadow-sm", style={'border-radius': '10px','height':'95.5%'})
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Tabs(id='heatmap-tabs', value=age_groups[0], children=[
                        dcc.Tab(label=group, value=group) for group in age_groups
                    ]),
                    html.Div(id='heatmap-container')
                ])
            ], className="mb-4"),
        ], width=9)
    ]),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Summary", className="card-title mb-4", style={'text-align': 'center', 'text-decoration': 'underline','font-weight':'bold'}),
                    dbc.Row(
                        [
                            dbc.Col(html.H5("Total Students", className="card-title mb-2", style={'text-align': 'left','font-weight':'bold'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='Total_Students', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    html.Hr(),
                    dbc.Row(
                        [
                            dbc.Col(html.H5("Overall GPA", className="card-title mt-2 mb-2", style={'text-align': 'left','font-weight':'bold'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='Overall_Gpa',className='mt-2', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    html.Hr(),
                    dbc.Row(
                        [
                            dbc.Col(html.H5("Course", className="card-title mt-2 mb-4", style={'text-align': 'left','font-weight':'bold'}), width="auto"),
                            dbc.Col(html.H5("Average GPA", className="card-title mt-2 mb-4", style={'text-align': 'right','font-weight':'bold'}), width="auto")
                        ],
                        className="d-flex justify-content-between"
                    ),
                    *[dbc.Row(
                        [
                            dbc.Col(html.H6(id=f'{i}-course', style={'text-align': 'left'}),className='mb-2', width="auto"),
                            dbc.Col(html.H6(id=f'{i}-gpa', style={'text-align': 'right'}), width="auto",className='mb-2', style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ) for i in range(1, 6)]
                ])
            ], className="mb-4")
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Sponosrship Distribution", className="card-title mb-2", style={'text-align': 'center', 'text-decoration': 'underline','font-weight':'bold'}),
                    dcc.Graph(id='age-distribution-pie', config={'displayModeBar': False})
                ])
            ], className="mb-3", style={'height': '95.5%'})
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Distribution in Courses", className="card-title mb-2", style={'text-align': 'center', 'text-decoration': 'underline','font-weight':'bold'}),
                    dcc.Graph(id='nationality-histogram')
                ])
            ], className="mb-3")
        ], width=6)
    ])
], fluid=True)

@app.callback(
    Output('heatmap-container', 'children'),
    [Output(f'{i}-gpa', 'children') for i in range(1, 6)],
    [Output(f'{i}-course', 'children') for i in range(1, 6)],
    Output('Total_Students', 'children'),
    Output('Overall_Gpa', 'children'),
    Output('age-distribution-pie', 'figure'),
    Output('nationality-histogram', 'figure'),
    Input('radio-buttons', 'value'),
    Input('Job-category', 'value'),
    Input('Gender', 'value'),
    Input('heatmap-tabs', 'value'),
    Input('x-axis', 'value'),
    Input('y-axis', 'value')
)
def update_graphs(selected_nationality, selected_job_categories, selected_gender, selected_tab, x_axis, y_axis):
    filtered_df = df

    filtered_df = filtered_df[filtered_df['Age Group'] == selected_tab]

    if selected_nationality != 'ALL':
        filtered_df = filtered_df[filtered_df['NATIONALITY'] == selected_nationality]
    
    if selected_job_categories != 'ALL':
        filtered_df = filtered_df[filtered_df['Job Category'].isin(selected_job_categories)]
        
    if selected_gender != 'ALL':
        filtered_df = filtered_df[filtered_df['GENDER'] == selected_gender]
    
    fig_heatmap = create_heatmap(filtered_df, x_axis, y_axis, f'(Age Group {selected_tab})')
    fig_age_distribution = create_age_distribution_pie(filtered_df)
    fig_nationality_histogram = create_nationality_histogram(filtered_df)

    short_forms_code = {
        1101: 'Diploma in Business Administration',
        2101: 'Certificate in HR Management',
        2102: 'Certificate in Digital Marketing',
        5112: 'Bachelor of Business Administration',
        5113: 'Master of Business Administration'
    }

    gpa_values = []
    courses = list(short_forms_code.values())
    for course in courses:
        gpa = round(filtered_df.loc[filtered_df['COURSE'] == course, 'GPA'].mean(), 2)
        gpa_values.append(gpa if pd.notna(gpa) else "N/A")

    # Find the highest and lowest GPAs
    numeric_gpas = [gpa for gpa in gpa_values if isinstance(gpa, (int, float))]
    max_gpa = max(numeric_gpas) if numeric_gpas else None
    min_gpa = min(numeric_gpas) if numeric_gpas else None

    # Create styled GPA and course outputs
    styled_gpas = []
    styled_courses = []
    for gpa, course in zip(gpa_values, courses):
        if gpa == "N/A":
            styled_gpas.append(html.Span("N/A"))
            styled_courses.append(html.Span(course))
        elif gpa == max_gpa:
            style = {'color': 'green', 'font-weight': 'bold'}
            styled_gpas.append(html.Span(gpa, style=style))
            styled_courses.append(html.Span(course, style=style))
        elif gpa == min_gpa:
            style = {'color': 'red', 'font-weight': 'bold'}
            styled_gpas.append(html.Span(gpa, style=style))
            styled_courses.append(html.Span(course, style=style))
        else:
            styled_gpas.append(html.Span(gpa))
            styled_courses.append(html.Span(course))

    Total_Students = filtered_df.shape[0]
    
    Overall_Gpa = round(filtered_df['GPA'].mean(), 2)
    
    return (
        dcc.Graph(id='heatmap-graph', figure=fig_heatmap),
        *styled_gpas,
        *styled_courses,
        Total_Students,
        Overall_Gpa,
        fig_age_distribution,
        fig_nationality_histogram
    )

if __name__ == '__main__':
    app.run_server(port=8123)